# Android Architecture

Application architecture is a way of designing classes and the relationship between them so that they are easier to work with and to update or maintain. Architecture implements the Seperation of Concerns principle which states that an application should be devidided into units, each with a single responsibillity. An application architecture devides an application into classes by seperating concerns.

### MVVM

![MVVM Architecture](https://miro.medium.com/max/3372/1*itYWsxQTfq7xTuvIMrVhYg.png)

#### UI Controller

The __UI Controller__ is responsible for any user interface related tasks, like displaying views or capturing user input. The UI controller executes all of the draw commands that puts the views on the screen

#### View Model

##### *What is a ViewModel*

Holds all of the data needed for the UI and prepare it for display through simple calculations and transformations. The View Model class also contains instances of the LiveData class, which is needed to communicate information from the ViewModel to the UI controller.

Another feature of the ViewModel is that it survives configuration changes, where fragments and views do not. This means that the current state of the data will survive and it reduces the need to rely on savedInstanceState


##### *Create a View Model*

View models are created for and associated with a single fragment, view, or activity.

1. Add Dependencies

Add the lifecycle dependency to the app gradle file

In [ ]:
dependencies {
    ...
    // Lifecycles
    implementation 'androidx.lifecycle:lifecycle-extensions:2.2.0'
    ...
}

2. Create a ViewModel Class

Subclass the ViewModel class. 

In [ ]:
package com.example.android.guesstheword.screens.game

import android.util.Log
import androidx.lifecycle.ViewModel

class GameViewModel: ViewModel() {
    init{
        Log.i("GameViewModel","GameViewModel created")
    }

    override fun onCleared() {
        super.onCleared()
        Log.i("GameViewModel","GameViewModel Destroyed")
    }


}

3. Associate UI Controller and ViewModel

The UI Controller is simply the fragment or activity class. Create an instance of the ViewModel inside of the fragment class using the viewModelProvider inside of the onCreateView() method

In [ ]:
viewModel = ViewModelProvider(this).get(GameViewModel::class.java)

##### *What is the LiveData class?*

The liveData class is an observable data holder class tha tis lifecycle-aware. The liveData object wraps around data and acts like a subject that is observed by an observer which then calls a specified method whenever the subject changes.

In the ViewModel, the LiveData class is used to automatically update the UIController, whenever state changes.

##### *Implementing LiveData class*

1. Declare LiveData variable in the ViewModel class and initialize it in the constructor

In [ ]:
    var word = MutableLiveData<String>()
    
    init{
        ...
        word.value = wordList.removeAt(0)
    }

2. Encapsulate the LiveData variable

By making it private and declaring a new unmutable LiveData variable that uses a backing field to create a getter method that will return a value

In [ ]:
private var _score = MutableLiveData<Int>()
    val score: LiveData<Int>
        get() = _score

3. Connect the Observer

In the Fragment class, connect the observer. The observer will observe the livedata object for any state changes, and then execute the specified function if it occurs

In [ ]:
viewModel.word.observe(viewLifecycleOwner,Observer{ newWord ->

        binding.wordText.text = newWord
        
})

##### *Creating a ViewModel Factory*

Sometimes a view model needs to be initialized with a value when it is created, and so a viewModel factory can be used to accomplish this. The following block of code is a fragment class that displays the final score at the end of the Game. 

In [ ]:
class ScoreFragment : Fragment() {

    override fun onCreateView(
            inflater: LayoutInflater,
            container: ViewGroup?,
            savedInstanceState: Bundle?
    ): View? {

        // Inflate view and obtain an instance of the binding class.
        val binding: ScoreFragmentBinding = DataBindingUtil.inflate(
                inflater,
                R.layout.score_fragment,
                container,
                false
        )

        // Get args using by navArgs property delegate
        val scoreFragmentArgs by navArgs<ScoreFragmentArgs>()
        binding.scoreText.text = scoreFragmentArgs.score.toString()
        binding.playAgainButton.setOnClickListener { onPlayAgain() }

        return binding.root
    }

    private fun onPlayAgain() {
        findNavController().navigate(ScoreFragmentDirections.actionRestart())
    }
}

To refactor this class into a UI controller and a ViewModel, we need to create a Factory that will initialize the viewModel with the Final score value on creation

1. Create the ViewModel Class

In the same folder as the fragment class, create the ViewModel class

In [ ]:
class ScoreViewModel(finalScore:Int): ViewModel() {

    var _finalScore:Int = 0

    init{
        _finalScore = finalScore
    }
}

2. Creat the ViewModel Factory

In the same folder as the fragment class, create the ViewModelFactory class

In [ ]:
class ScoreViewModelFactory(private val finalScore: Int) : ViewModelProvider.Factory {
    
    //The create method returns the viewModel
    override fun <T : ViewModel?> create(modelClass: Class<T>): T {
        if (modelClass.isAssignableFrom(ScoreViewModel::class.java)) {
            // TODO Construct and return the ScoreViewModel
            return ScoreViewModel(finalScore) as T
        }
        throw IllegalArgumentException("Unknown ViewModel class")
    }
}

3. Initialize the Factory in the UIController Fragment

In [ ]:
class ScoreFragment : Fragment() {
    
    //Declare the viewModel variables
    private lateinit var viewModel: ScoreViewModel
    private lateinit var viewModelFactory:ScoreViewModelFactory

    override fun onCreateView(
            inflater: LayoutInflater,
            container: ViewGroup?,
            savedInstanceState: Bundle?
    ): View? {

        // Inflate view and obtain an instance of the binding class.
        val binding: ScoreFragmentBinding = DataBindingUtil.inflate(
                inflater,
                R.layout.score_fragment,
                container,
                false
        )



        // Get args using by navArgs property delegate
        val scoreFragmentArgs by navArgs<ScoreFragmentArgs>()


        //Create Factory by passing in the score
        viewModelFactory = ScoreViewModelFactory(scoreFragmentArgs.score)
        
        //Pass the Factory into the viewModelProvider to create
        //the viewModel
        viewModel = ViewModelProvider(this,viewModelFactory).get(ScoreViewModel::class.java)

        binding.scoreText.text = viewModel._finalScore.toString()
        binding.playAgainButton.setOnClickListener { onPlayAgain() }


        return binding.root
    }

    private fun onPlayAgain() {
        findNavController().navigate(ScoreFragmentDirections.actionRestart())
    }

##### *Bypassing the UIController by binding to the ViewModel*

1. Add Data tag to layout file

Create a data variable for the viewModel you want to add.

In [ ]:
<layout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools">

    <data>
        <variable
            name="gameViewModel"
            type="com.example.android.guesstheword.screens.game.GameViewModel" />
    </data>

    <androidx.constraintlayout.widget.ConstraintLayout
        android:id="@+id/game_layout"
        android:layout_width="match_parent"
        android:layout_height="match_parent"
        tools:context=".screens.game.GameFragment">



        <TextView
            android:id="@+id/word_is_text"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_marginBottom="16dp"
            android:fontFamily="sans-serif"
            android:text="@string/word_is"
            android:textColor="@color/black_text_color"
            android:textSize="14sp"
            android:textStyle="normal"
            app:layout_constraintBottom_toTopOf="@+id/word_text"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintHorizontal_bias="0.5"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toTopOf="parent"
            app:layout_constraintVertical_chainStyle="packed" />

        <TextView
            android:id="@+id/word_text"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:animateLayoutChanges="true"
            android:fontFamily="sans-serif"
            android:textAppearance="@style/TextAppearance.AppCompat.Headline"
            android:textColor="@color/black_text_color"
            android:textSize="34sp"
            android:textStyle="normal"
            app:layout_constraintBottom_toTopOf="@+id/score_text"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintHorizontal_bias="0.5"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/word_is_text"
            app:layout_constraintVertical_chainStyle="packed"
            tools:text="&quot;Tuna&quot;" />

        <TextView
            android:id="@+id/timer_text"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_marginStart="8dp"
            android:layout_marginEnd="8dp"
            android:layout_marginBottom="8dp"
            android:fontFamily="sans-serif"
            android:textColor="@color/grey_text_color"
            android:textSize="14sp"
            android:textStyle="normal"
            app:layout_constraintBottom_toTopOf="@+id/score_text"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toStartOf="parent"
            tools:text="0:00" />

        <TextView
            android:id="@+id/score_text"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_marginStart="8dp"
            android:layout_marginEnd="8dp"
            android:fontFamily="sans-serif"
            android:textColor="@color/grey_text_color"
            android:textSize="14sp"
            android:textStyle="normal"
            app:layout_constraintBottom_toTopOf="@+id/guideline"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toStartOf="parent"
            tools:text="Score: 2" />

        <Button
            android:id="@+id/skip_button"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_marginStart="16dp"
            android:text="@string/skip"
            android:theme="@style/SkipButton"
            app:layout_constraintBottom_toBottomOf="parent"
            app:layout_constraintEnd_toStartOf="@+id/correct_button"
            app:layout_constraintHorizontal_bias="0.5"
            app:layout_constraintHorizontal_chainStyle="spread_inside"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toTopOf="@+id/guideline" />

        <androidx.constraintlayout.widget.Guideline
            android:id="@+id/guideline"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:orientation="horizontal"
            app:layout_constraintGuide_end="96dp" />

        <Button
            android:id="@+id/correct_button"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_marginEnd="16dp"
            android:text="@string/got_it"
            android:theme="@style/GoButton"
            android:onClick="@{() -> gameViewModel.onSkip()}"
            app:layout_constraintBottom_toBottomOf="parent"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintHorizontal_bias="0.5"
            app:layout_constraintStart_toEndOf="@+id/skip_button"
            app:layout_constraintTop_toTopOf="@+id/guideline" />

    </androidx.constraintlayout.widget.ConstraintLayout>
</layout>

2. Add viewModel to binding

In the fragment class, just after the binding has be initilized, add the ViewModel to the binding variable created above

In [ ]:
binding.gameViewModel = viewModel

3. Add onClick listeners directly to component

In the layout file add the onClick listener directly to the component. Now when the button is clicked, it will call the viewModel directly, so no need for the UIController to set or handle onClick events.

In [ ]:
 <Button
            android:id="@+id/correct_button"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:layout_marginEnd="16dp"
            android:text="@string/got_it"
            android:theme="@style/GoButton"
            ...
            android:onClick="@{() -> gameViewModel.onCorrect()}"
            ...
            app:layout_constraintBottom_toBottomOf="parent"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintHorizontal_bias="0.5"
            app:layout_constraintStart_toEndOf="@+id/skip_button"
            app:layout_constraintTop_toTopOf="@+id/guideline" />

##### *LiveData binding the ViewModel*

This is done to automatically update information fields like textviews, etc

1. Bind viewModel and set the life cycle owner

NOTE: remember to first add the data variable directly to the layout file.

In [ ]:
binding.gameViewModel = viewModel
binding.setLifecycleOwner(this)

2. Add the binding field to the view

This will automatically keep the UI in sync with the live data without the need for setting up an observer in the fragment class

In [ ]:
<TextView
            android:id="@+id/word_text"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:animateLayoutChanges="true"
            android:fontFamily="sans-serif"
            android:textAppearance="@style/TextAppearance.AppCompat.Headline"
            android:textColor="@color/black_text_color"
            android:textSize="34sp"
            android:textStyle="normal"
            ...
            android:text="@{gameViewModel.word}"
            ...
            app:layout_constraintBottom_toTopOf="@+id/score_text"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintHorizontal_bias="0.5"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/word_is_text"
            app:layout_constraintVertical_chainStyle="packed"
            tools:text="&quot;Tuna&quot;" />